In [1]:
#import some libraries
import time
import requests,re
from bs4 import BeautifulSoup
import json
import pymysql as pymysql
import sys
import os
import mysql.connector
import csv
import codecs

Web-scraping: Which ipad4 sellers advertise/sponsor on eBay

a) For the first 10 pages of 100 items/page, save all the URLs of sponsored items' pages to the file "sponsored.txt"  and all the URLs of non-sponsored items' pages to the file "non-sponsored.txt" in the same directory as your code. (One URL per line in each file)

In [47]:
# Define a function to get information on the page
def get_page(url_ebay):
    time.sleep(10)
    url_ebay= url_ebay
    header={'user-agent':'Mozilla/5.0'}
    r=requests.get(url_ebay,headers=header)
    r.raise_for_status()
    r.encoding=r.apparent_encoding
    return r.text

In [50]:
# Define a function to save pages as html file
def save_page(text,page_num):
    HTML_name = "ipad-4"+page_num+".html"
    message=text
    f = open(HTML_name,'w')
    f.write(message)
    f.close()

In [51]:
# Begin scraping and save the html files
os.chdir('/Users/calvin/Desktop/Github page/Webscraping Project')
for i in range(1,11):
    url_ebay='https://www.ebay.com/sch/i.html?_from=R40&_nkw=ipad+4&_sacat=0&LH_TitleDesc=0&LH_BIN=1&_pgn='+str(i)
    content=get_page(url_ebay)
    if i < 10:
        save_page(content,"0"+str(i))
    else:
        save_page(content,str(i))

In [52]:
# Create two txt files
os.chdir('/Users/calvin/Desktop/Github page/Webscraping Project')
f_spon = open("sponsored.txt", "x")
f_non = open("non-sponsored.txt", "x")

In [53]:
# Save all the urls to the txt files
for page in range(1,11):
    if page<10:
        filename="ipad-40"+str(page)+".html"
    else:
        filename="ipad-4"+str(page)+".html"
    with open(filename, 'r') as file:
        text = file.read()
        soup = BeautifulSoup(text, 'html.parser') 
        items = soup.select("#srp-river-results > ul > li.s-item")
        for item in items:
                a = item.find("a")
                if not a:
                    continue         
                text = item.find("span").get_text()
                if re.search('.*S.*P.*O.*N.*S.*O.*R.*E.*D.*',text):  
                    url_simplify = re.sub("(.*)\\?.*", r"\1", a['href'])
                    f_spon = open("sponsored.txt", "a")
                    f_spon.write(url_simplify)
                    f_spon.write("\n")
                    f_spon.close()
                else:
                    url_non_simplify = re.sub("(.*)\\?.*", r"\1", a['href'])
                    f_nonspon = open("non-sponsored.txt", "a")
                    f_nonspon.write(url_non_simplify)
                    f_nonspon.write("\n")
                    f_nonspon.close()                   

b) Create two folders in the same directory as your code and name them "sponsored" and "non-sponsored". Write a program that opens the two files in (a) and downloads each of the pages (URLs) into the folders "sponsored" and "non-sponsored". Each file should be named as "<item-id>.htm" where you replace "item-id" with the ID of the item you are saving. E.g., "264616053293.htm" for the item with ID "264616053293". Note it is always good to put a 2-second pause between queries. Make sure to catch an error and continue if your query runs into problems connecting to eBay (e.g., if your internet is down for 5 seconds, you don't want your entire code to crash).

In [54]:
# Get all the urls for sponsored item
os.chdir('/Users/calvin/Desktop/Github page/Webscraping Project')
txtFile = open('sponsored.txt','r')
txtpage = txtFile.read()
txtFile.close()

In [55]:
# Save all the sponsored urls to a list
x = re.findall(r'http.*', txtpage)
len(x)
# There are 108 items in the list

108

In [56]:
# Save the txt files in sponsored folder
p1 = re.compile(r'.*([0-9]{12})')
sponsor_count = list()
for i in range(1,108):
    time.sleep(2)
    os.chdir('/Users/calvin/Desktop/Github page/Webscraping Project/sponsored')
    link = x[i]
    filename = re.findall(p1, link)
    filename = ''.join(map(str, filename)) 
    with open(filename + '.txt', 'w') as f:
        f.write(link)
        f.close
    sponsor_count.append(filename) # Create a list containing all the filenames.

In [57]:
# Get all the urls for non-sponsored item
os.chdir('/Users/calvin/Desktop/Github page/Webscraping Project')
txtFile_non = open('non-sponsored.txt','r')
txtpage_non = txtFile_non.read()
txtFile_non.close()

In [58]:
# Save all the non-sponsored urls to a list
y = re.findall(r'http.*', txtpage_non)
len(y)
# There are 522 items in the list

522

In [59]:
# Save the txt files in non-sponsored folder
nonsponsor_count = list()
for j in range(1,522):
    time.sleep(2)
    os.chdir('/Users/calvin/Desktop/Github page/Webscraping Project/non-sponsored')
    link_non = y[j]
    filename_non = re.findall(p1, link_non)
    filename_non = ''.join(map(str, filename_non)) 
    with open(filename_non + '.txt', 'w') as f:
        f.write(link_non)
        f.close
    nonsponsor_count.append(filename_non) # Create a list containing all the filenames.

c) Write a separate piece of code that loops through the pages you downloaded in (b) and opens and parses them into a Python or Java xxxxsoup-object. Identify and select:

seller name, seller score, item price, # items sold, best offer available, title, returns allowed, shipping price, condition (e.g., used, new, like new, seller refurbished, ...).

In your code, highlight the selector command you choose to obtain each element using comments.

**This is the section used to obtain each element**

In [84]:
# Define a function to convert the price
def price_convert():
    price  = html_soup.find("span", {"class": "notranslate"}).text
    p_price = re.compile(r'\$([0-9]+.[0-9]+)') # $ sign must be with escape \, and should select the price only
    price_num = re.findall(p_price, price)
    price_num = ''.join(map(str, price_num)) 
    price_num = re.sub('\.','',price_num)
    price_num = re.sub(',','',price_num)
    if price_num:
        price_num = int(price_num)
    else:
        price_num = 0
    return(price_num)

# Define a function to get the seller score
def get_score():
    score = html_soup.find("span", {"class": "mbg-l"}).text
    score = re.findall('([0-9]+)', score)
    score = ''.join(map(str,score))
    score = int(score)
    return(score)

# Define a function to get the item_sold
def get_item_sold():
    item_content = html_soup.find("span", {"class": "soldwithfeedback"})
    if item_content:
        item_sold = html_soup.find("span", {"class": "soldwithfeedback"}).text
        item_sold = re.findall('([0-9]+)\ssold', item_sold)
        item_sold = ''.join(map(str,item_sold))
        item_sold = int(item_sold)
    else:
        item_sold = 0
    return(item_sold)
    

# Define a function to get the title
def get_title():
    title = html_soup.find('h1').text 
    title = re.findall('Details about\s(.*)', title)
    title = ''.join(map(str,title))
    title = re.sub(',','',title)
    title = re.sub('"','',title)
    return(title)

# Define a function get the return policy
def get_return():
    returns_content = html_soup.find("span", {"class": "w2b-sgl"})
    if returns_content:
        returns_text = html_soup.find("span", {"class": "w2b-sgl"}).text
        if returns_text:
            if returns_text == 'Returns accepted':
                returns = 'Accepted'
            else: 
                returns = 'Notaccepted'
    else:
        returns = 'Notclear'
    return(returns)

# Define a function to get the bestoffer price
def get_bestoffer():
    bestoffer = html_soup.find("div", {"class": "vi-bbox-dspn  u-flL w29 vi-price"})
    if bestoffer:
        if bestoffer.text == ' ':
            bestoffer = 0
        else:
            bestoffer = int(bestoffer.text)
    else:
        bestoffer = 0
    
    return(bestoffer)

# Define a function to get the condition
def get_condition():
    conditioncontent = html_soup.find("div", {"class": "u-flL condText"})
    if conditioncontent:
        condition = html_soup.find("div", {"class": "u-flL condText"}).text
    else:
        condition = 'Notclear'
    
    return(condition)

In [61]:
# Create a txt file to store all sponsored items information
os.chdir('/Users/calvin/Desktop/Github page/Webscraping Project/sponsored')
sqldata = open("sqldata.txt", "x")

In [77]:
# Get the url for sponosored items
os.chdir('/Users/calvin/Desktop/Github page/Webscraping Project/sponsored')
for i in range(0,106):
    file_count = sponsor_count[i]

    with open(file_count + '.txt', 'r') as f:
        content = f.read()
        f.close()

    # Go to the url and parse them
    header={'user-agent':'Mozilla/5.0'}
    response = requests.get(content,headers=header)
    # save the text of the html file
    html_soup = BeautifulSoup(response.text, 'html.parser')
    name = html_soup.find("span", {"class": "mbg-nw"}).text
    score = get_score()
    price_cent = price_convert()
    item_sold = get_item_sold()
    bestoffer = get_bestoffer()
    title = get_title()
    returns = get_return()
    #shipping_price = get_shippingprice()
    condition = get_condition()

    with open('sqldata.txt','a') as f:
        writer = csv.writer(f,delimiter=' ')
        writer.writerow([name,',',score,',',price_cent,',',item_sold,',', bestoffer,',',title,',',returns,',', condition,',','Yes'])
        f.close()

In [ ]:
# Create a txt file to store all non-sponsored items information
os.chdir('/Users/calvin/Desktop/Github page/Webscraping Project/non-sponsored')
open("nonsponsoredsqldata.txt", "x")

In [85]:
# Get the url for nonsponosored items
os.chdir('/Users/calvin/Desktop/Github page/Webscraping Project/non-sponsored')
for i in range(0,521):
    file_count = nonsponsor_count[i]

    with open(file_count + '.txt', 'r') as f:
        content = f.read()
        f.close()

    # Go to the url and parse them
    header={'user-agent':'Mozilla/5.0'}
    response = requests.get(content,headers=header)
    # save the text of the html file
    html_soup = BeautifulSoup(response.text, 'html.parser')
    name = html_soup.find("span", {"class": "mbg-nw"}).text
    score = get_score()
    price_cent = price_convert()
    item_sold = get_item_sold()
    bestoffer = get_bestoffer()
    title = get_title()
    returns = get_return()
    #shipping_price = get_shippingprice() # Unable to identify the shipping price
    condition = get_condition()

    with open('nonsponsoredsqldata.txt','a') as f:
        writer = csv.writer(f,delimiter=' ')
        writer.writerow([name,',',score,',',price_cent,',',item_sold,',', bestoffer,',',title,',',returns,',', condition,',','No'])
        f.close()

d) Use your code script to connect to SQL (either MySQL, MariaDB, or SQLite. Do NOT use SQL GUI or command terminal). Create a database and name it "eBay". Save the information of items in (c)  into a single table named "eBay_items" (You are allowed to use only one table). This table should contain both sponsored and non-sponsored information and have a column that specifies which item is sponsored/non-sponsored.  If an item misses ANY of the information in (c), you should insert that missing value as NULL into the table. Convert any price (item price and shipping price) into a "dollar-cent" format (e.g., convert $12.34 into 1234 and $12 into 1200. Make sure the two least significant digits are cents. If an item does not include cents in the price, insert zeros.) and insert the price as INT into the table.

In [11]:
# Create database
conn=pymysql.connect(host='localhost',user='root', passwd = '') # Have to keep the pw, without it the code doesn't work on my computer.
cursor = conn.cursor()
db_name = 'eBay'
cursor.execute('DROP DATABASE IF EXISTS %s' %db_name)
cursor.execute('CREATE DATABASE IF NOT EXISTS %s' %db_name)
cursor.execute('USE eBay')
table_name = 'eBay_items'
cursor.execute('DROP TABLE IF EXISTS %s' %table_name)
cursor.execute('CREATE TABLE %s(seller varchar(2000), score INT, item_price INT, item_sold INT, bestoffer INT, title TEXT, returns varchar(2000), conditions varchar(2000), sponsored varchar(2000))'%table_name)

0

In [12]:
# Insert sponsored data into the SQL table
os.chdir('/Users/calvin/Desktop/Github page/Webscraping Project/sponsored')

def get_conn():
  conn = pymysql.connect(host='localhost', user='root', db='eBay', charset='utf8')
  return conn

def insert(cur, sql, args):
  cur.execute(sql, args)
  
def read_csv_to_mysql(filename):
  with codecs.open(filename=filename, mode='r', encoding='utf-8') as f:
    reader = csv.reader(f)
    #head = next(reader)
    conn = get_conn()
    cur = conn.cursor()
    sql = 'insert into eBay_items values(%s, %s, %s, %s, %s, %s, %s, %s, %s)'
    for item in reader:
      if item[1] is None or item[1] == '':
        continue
      args = tuple(item)
      print(args)
      insert(cur, sql=sql, args=args)
    conn.commit()
    cur.close()
    conn.close()

def query_all(cur, sql, args):
  cur.execute(sql, args)
  return cur.fetchall()

def read_mysql_to_csv(filename):
  with codecs.open(filename=filename, mode='w', encoding='utf-8') as f:
    write = csv.writer(f, dialect='excel')
    conn = get_conn()
    cur = conn.cursor()
    sql = 'select * from tb_csv'
    results = query_all(cur=cur, sql=sql, args=None)
    for result in results:
      write.writerow(result)

if __name__ == '__main__':
  read_csv_to_mysql('sqldata.txt')

('save_smart ', ' 17491 ', ' 4999 ', ' 33 ', ' 0 ', ' " \xa0Apple iPad 4 Tab 4th Gen 16GB WiFi 9.7 MD510LL/A Touchscreen Working Tablet" ', ' Notaccepted ', ' Used ', ' Yes')
('ltdealsdirect ', ' 1038 ', ' 9999 ', ' 708 ', ' 0 ', ' " \xa0Apple iPad 4 (4th Gen) Wi-Fi + Cellular - 16GB 32GB 64GB 128GB - Black - White" ', ' Accepted ', ' "Manufacturer refurbished" ', ' Yes')
('itsworthmore ', ' 32674 ', ' 7299 ', ' 372 ', ' 0 ', ' " \xa0Apple iPad 4 16GB 32GB 64GB 128GB WiFi OR Cellular Black / White" ', ' Notaccepted ', ' Used ', ' Yes')
('eazytrade84 ', ' 84735 ', ' 12985 ', ' 677 ', ' 0 ', ' " \xa0Apple iPad 4th Gen Retina Display 32GB Wi-Fi 9.7in - BLACK - GRADE A (R)" ', ' Accepted ', ' "Manufacturer refurbished" ', ' Yes')
('2010thisisit ', ' 5299 ', ' 9999 ', ' 122 ', ' 0 ', ' " \xa0Apple iPad 4 16GB Wi-Fi 9.7in - Black  WIFI ((Retina display ))" ', ' Notaccepted ', ' "Seller refurbished" ', ' Yes')
('eazytrade84 ', ' 84735 ', ' 10985 ', ' 426 ', ' 0 ', ' " \xa0Apple iPad 4 WiFi + 

In [13]:
# Insert non-sponsored data into the SQL table
os.chdir('/Users/calvin/Desktop/Github page/Webscraping Project/non-sponsored')
if __name__ == '__main__':
  read_csv_to_mysql('nonsponsoredsqldata.txt')

('meiestore ', ' 31745 ', ' 6800 ', ' 77 ', ' 0 ', ' " \xa0Apple iPad 2/3/4 Generation 16GB 32GB 64GB Wi-Fi 9.7in" ', ' Notaccepted ', ' Used ', ' No')
('bigdeals ', ' 382268 ', ' 23999 ', ' 675 ', ' 0 ', ' " \xa0Apple iPad 4 - 4th Generation 9.7 with Retina Display 16GB 32GB 64GB WIFI" ', ' Notaccepted ', ' "Manufacturer refurbished" ', ' No')
('marina1874 ', ' 4516 ', ' 7900 ', ' 0 ', ' 0 ', ' " \xa0Apple iPad 4 (4th Generation) 9.7 Retina Display Black/White" ', ' Accepted ', ' Used ', ' No')
('techgallery1 ', ' 2171 ', ' 8995 ', ' 350 ', ' 0 ', ' " \xa0NEW Apple iPad 2 3 4 Mini Air Air 2 Pro - WiFi or Cell - 16GB 32GB 64GB 128GB " ', ' Notaccepted ', ' "Open box" ', ' No')
('marina1874 ', ' 4516 ', ' 6500 ', ' 753 ', ' 0 ', ' " \xa0Apple iPad Mini 1/2/3/4 WIFI 7.9 Display" ', ' Accepted ', ' Used ', ' No')
('sportssurplus10 ', ' 4232 ', ' 7299 ', ' 0 ', ' 0 ', ' " \xa0Apple iPad 4th Gen. - 16GB - Wi-Fi Only - 9.7in - Black - (MD510LL/A)" ', ' Notaccepted ', ' Used ', ' No')
('sport